In [8]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
from dotenv import load_dotenv
import google.generativeai as genai
from minsearch import Index

In [5]:
load_dotenv()
genai.configure(api_key=os.environ["API_KEY"])

In [6]:
# Prepare dataset
# using gotquestions dataset

import json

with open("datasets/questions_answer.json", 'rt') as f_in:
    data = json.load(f_in)

print(data[0])

{'category': 'Questions about God', 'sub.category': 'The Nature of God', 'question': 'What is God like?', 'answer': 'Answer\nEvery culture in the history of the world has had some concept of what God is like. Some have assumed that God is in control of the weather and have made images of a storm god throwing lightning bolts around (Baal worship in Canaan). Some have assumed that God is very powerful, and so they worshiped the most powerful thing they could see, the sun (Ra worship in Egypt). Others have assumed that God is everywhere and therefore have worshiped everything (pantheism in Stoic philosophy). Some have assumed that God is unknowable and have turned to agnosticism or, just to cover their bases, have worshiped “An Unknown God” (Acts 17:23).\nThe problem with each of these assumptions is that they only get part of the picture of who God is. Yes, God is in control of the weather, but He is also in control of so much more. He is powerful, but much more powerful than the sun. He

In [ ]:

index = Index(
    text_fields = ["question", "answer", "sub.category"],
    keyword_fields = ["category"]
)

index.fit(data)

In [ ]:
def search(query):
# filter_dict = {"category": "Questions about God"}
    boost_dict = {"question": 3, "answer": 1, "sub.category": 1}

    return index.search(query, boost_dict, num_results=5)

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a spiritual assistant, your goal is to answer questions about christian spirituality like God, Jesus, Holy Spirit, and Life as a believer of this spirituality.
    Answer the QUESTIONS based on the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't match or contain the answer, give NONE as the response.
    Answer the questions as if you are a spiritual assistant. Don't provide personal opinions or beliefs.
    Don't format your answer quoting the context, just answer the question.
    
    QUESTION : {question}
    
    CONTEXT: {context}
    
    """.strip()

    context = ""
    for doc in search_results:
        context = f'{context}sub.category: {doc["sub.category"]} \nquestion: {doc["question"]} \nanswer: {doc["answer"]}\n\n'

    return prompt_template.format(question= query, context=search_results).strip()

In [30]:
def llm(prompt):
    model = genai.GenerativeModel("gemini-1.5-pro")
    result = model.generate_content(prompt)
    return result.text
    

In [32]:
def rag(query):
    results = search(query)
    prompt = build_prompt(query, results)
    return llm(prompt)

In [36]:
rag("Jesus loves me")

'The contexts provided discuss Jesus\' love for children, the timeline of his life, his divinity, and whether he was married.  None of them explicitly states "Jesus loves me." While the contexts affirm Jesus\' love for all people, including children, they don\'t contain the specific phrase requested. Therefore, the answer is NONE.\n'